In [ ]:
import pandas as pd
import math

### Load 
df = pd.read_csv("/home/mouren/Data/valid_exp/summary_count_exonhancers_bilan.tsv", sep="\t", header=0)
df = df[~df['Log2FC'].str.contains('#', na=False)] # Remove rows with no log2FC values 

#Log2FC of merged replicates and standard deviation
df["Log2FC"] = df["Log2FC"].str.replace(',', '.')
df["Input_STD"] = df["Input_STD"].str.replace(',', '.')

df["Input_STD"] = pd.to_numeric(df["Input_STD"])
df["Log2FC"] = pd.to_numeric(df["Log2FC"])

#Input normalisé mergé
df["CPM_AVO4Exonhmaxi4"] = df["CPM_AVO4Exonhmaxi4"].str.replace(',', '.') 
df["CPM_AVO4Exonhmaxi4"] = pd.to_numeric(df["CPM_AVO4Exonhmaxi4"])

#Input normalisé 
df["CPM_AVO1ExonhDNA1"] = df["CPM_AVO1ExonhDNA1"].str.replace(',', '.')
df["CPM_AVO2ExonhDNA2"] = df["CPM_AVO2ExonhDNA2"].str.replace(',', '.')
df["CPM_AVO3ExonhDNA3"] = df["CPM_AVO3ExonhDNA3"].str.replace(',', '.')

df["CPM_AVO1ExonhDNA1"] = pd.to_numeric(df["CPM_AVO1ExonhDNA1"])
df["CPM_AVO2ExonhDNA2"] = pd.to_numeric(df["CPM_AVO2ExonhDNA2"])
df["CPM_AVO3ExonhDNA3"] = pd.to_numeric(df["CPM_AVO3ExonhDNA3"])

#Output normalisé mergé
df["CPM_AVO7ExonhcDNA3"] = df["CPM_AVO7ExonhcDNA3"].str.replace(',', '.')
df["CPM_AVO8ExonhcDNA4"] = df["CPM_AVO8ExonhcDNA4"].str.replace(',', '.')
df["CPM_AVO9ExonhcDNA5"] = df["CPM_AVO9ExonhcDNA5"].str.replace(',', '.')

df["CPM_AVO7ExonhcDNA3"] = pd.to_numeric(df["CPM_AVO7ExonhcDNA3"])
df["CPM_AVO8ExonhcDNA4"] = pd.to_numeric(df["CPM_AVO8ExonhcDNA4"])
df["CPM_AVO9ExonhcDNA5"] = pd.to_numeric(df["CPM_AVO9ExonhcDNA5"])

## To filter
df["count_sequence_AVO4Exonhmaxi4"] = pd.to_numeric(df["count_sequence_AVO4Exonhmaxi4"])

# removes input count reads less than 500 and standard deviation above 20
filtered_df = df[(df['count_sequence_AVO4Exonhmaxi4'] >= 500) & (df['Input_STD'] <= 20) & (df['Category2'].isin(["EEK","EESNP","EEMULTISNP"]))]

In [ ]:
### Make a dic matching each snp sequence with its ref
filtered_df['Name'] = filtered_df['ID'].apply(lambda x: '_'.join(x.split('_')[:3]))

df_no_snp = filtered_df[(filtered_df['Category2'] == "EEK")]
df_snp = filtered_df[(filtered_df['Category2'] == "EESNP")]
df_multisnp = filtered_df[(filtered_df['Category2'] == "EEMULTISNP")]

list_snp = []
for index,row in df_snp.iterrows():
    #find sample without snp
    matching_rows = df_no_snp.loc[df_no_snp['Name'] == row['Name']]
 
    if matching_rows.empty: #row without snp has been cut off because of input standarad deviation or not enough reads
        continue
    
    list_snp.append([row["ID"],matching_rows["ID"].iloc[0]])

for index,row in df_multisnp.iterrows():
    matching_rows = df_no_snp.loc[df_no_snp['Name'] == row['Name']]

    if matching_rows.empty: #row without snp has been cut off because of input standarad deviation or not enough reads
        continue
    
    list_snp.append([row["ID"],matching_rows["ID"].iloc[0]])

df_tosave = pd.DataFrame(list_snp)

df_tosave.to_csv("/home/mouren/Data/valid_exp/stat/seq_snp_ref_id.tsv",sep="\t",header=False,index=False)

/tmp/ipykernel_1745069/2163807241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Name'] = filtered_df['ID'].apply(lambda x: '_'.join(x.split('_')[:3]))


In [6]:
### Make a dic matching each single snp sequence with its multi ref
filtered_df['Name'] = filtered_df['ID'].apply(lambda x: '_'.join(x.split('_')[:3]))

df_snp = filtered_df[(filtered_df['Category2'] == "EESNP")]
df_multisnp = filtered_df[(filtered_df['Category2'] == "EEMULTISNP")]

list_snp = []
for index,row in df_snp.iterrows():
    #find sample multi snp
    matching_rows = df_multisnp.loc[df_multisnp['Name'] == row['Name']]
 
    if matching_rows.empty: #row with multi snp don't exist or has been cut off because of input standarad deviation or not enough reads
        continue
    
    list_snp.append([row["ID"],matching_rows["ID"].iloc[0]])

df_tosave = pd.DataFrame(list_snp)
df_tosave.to_csv("/home/mouren/Data/valid_exp/stat/seq_EESNP_MULTISEQ_id.tsv",sep="\t",header=False,index=False)

/tmp/ipykernel_1745069/3137204533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Name'] = filtered_df['ID'].apply(lambda x: '_'.join(x.split('_')[:3]))
